# Rede Neural Perceptron de 3 camadas - Exercícios de fixação

### Bibliotecas

In [1]:
import numpy as np
import pandas as pd
import random
import csv

### Datasets

In [2]:
# Função AND
dt_AND = np.array([[0, 0, 0],
                   [0, 1, 0],
                   [1, 0, 0],
                   [1, 1, 1]])

# Função OR
dt_OR = np.array([[0, 0, 0],
                  [0, 1, 1],
                  [1, 0, 1],
                  [1, 1, 1]])

# Função XOR
dt_XOR = np.array([[0, 0, 0],
                   [0, 1, 1],
                   [1, 0, 1],
                   [1, 1, 0]])

# Função LOGICA
dt_LOGICA = np.array([[0, 0, 0, 0],
                   [0, 0, 1, 1],
                   [0, 1, 0, 1],
                   [0, 1, 1, 0],
                   [1, 0, 0, 0],
                   [1, 0, 1, 1],
                   [1, 1, 0, 1],
                   [1, 1, 1, 0]])

### Função de Ativação

In [3]:
def sigmoid(u):
    return 1.0 / (1.0 + np.exp(-u))

### Métodos de inicialização de pesos

In [4]:
def aleatorio_uniforme(n_entrada, n_oculta1, n_oculta2, n_saida):
    random.seed(30)
    w1 = np.zeros((n_entrada, n_oculta1))
    w2 = np.zeros((n_oculta1, n_oculta2)) #Peso INT1->INT2
    w3 = np.zeros((n_oculta2, n_saida))
    for i in range(0, n_entrada):
        for j in range(1, n_oculta1):
            aleatorio = random.randint(1, 101)
            w1[i,j] = (1.0 - 2.0 * aleatorio) / 100.0
    for j in range(0, n_oculta1):
        for k in range(0, n_oculta2):
            aleatorio = random.randint(1, 101)
            w2[j,k] = (1.0 - 2.0*aleatorio)/100.0
    for j in range(0, n_oculta2):
        for k in range(0, n_saida):
            aleatorio = random.randint(1, 101)
            w3[j,k] = (1.0 - 2.0 * aleatorio) / 100.0
    return w1, w2, w3

### Classe Percetron Multicamada

In [5]:
class MLP:
    def __init__(self, n_entrada, n_oculta, n_saida, inicializa_pesos, funcao_ativacao, nome_dt):
        self.n_entrada = n_entrada
        self.n_oculta1 = n_oculta[0]
        self.n_oculta2 = n_oculta[1]
        self.n_saida = n_saida
        self.inicializa_pesos = inicializa_pesos
        self.w1 = self.w2 = self.w3 = np.array([])
        self.delta1 = self.delta2 = self.delta3 = np.array([])
        self.funcao_ativacao = funcao_ativacao
        self.nome_dt = nome_dt
        self.erro_max = 0.0
        self.epocas = 0
        self.qtd_acertos = 0
        #self.bias = 0.1
    
    def intermediaria1(self, x):
        h1 = np.zeros(self.n_oculta1)
        h1[0] = 1.0
        for j in range(1, self.n_oculta1):
            somatorio = 0.0
            for i in range(0, self.n_entrada):
                somatorio = somatorio + x[i] * self.w1[i,j]
            h1[j] = self.funcao_ativacao(somatorio)
        return h1

    def intermediaria2(self, h1):
        h2 = np.zeros(self.n_oculta2)
        h2[0] = 1.0
        for j in range(1, self.n_oculta2):
            somatorio = 0.0
            for i in range(0, self.n_oculta1):
                somatorio = somatorio + h1[i] * self.w2[i,j]
            h2[j] = self.funcao_ativacao(somatorio)
        return h2

    def saida(self, h2):
        o = np.zeros(self.n_saida)
        for j in range(0, self.n_saida):
            somatorio = 0.0
            for i in range(0, self.n_oculta2):
                somatorio = somatorio + h2[i] * self.w3[i,j]
            somatorio = -somatorio
            o[j] = 1.0 / (1.0 + np.exp(somatorio))
        return o

    def erro_saida(self, o, y):
        somatorio = 0.0
        y = [y]
        for i in range(self.n_saida):
            somatorio = somatorio + (y[i] - o[i])*(y[i] - o[i])
        erro = 0.5 * somatorio
        return erro
        
    def erro3(self, o, y):
        y = [y]
        delta3 = np.zeros(self.n_saida)
        for i in range(self.n_saida):
            delta3[i] = o[i] * (1.0 - o[i]) * (y[i] - o[i])
        return delta3
   
    def erro2(self, h2, delta3):
        #delta2 = np.zeros(self.n_oculta2-1)
        delta2 = np.zeros(self.n_oculta2)
        for i in range(1, self.n_oculta2):
            somatorio = 0.0
            for j in range(0, self.n_saida):
                somatorio = somatorio + self.w3[i][j] * delta3[j]
            delta2[i] = h2[i] * (1.0 - h2[i]) * somatorio
        return delta2
    
    def erro1(self, h1, delta2):
        #delta1 = np.zeros(self.n_oculta1-1)
        delta1 = np.zeros(self.n_oculta1)
        for i in range(1, self.n_oculta1):
            somatorio = 0.0
            for j in range(1, self.n_oculta2):
                somatorio = somatorio + self.w2[i][j] * delta2[j]
            delta1[i] = h1[i] * (1.0 - h1[i]) * somatorio
        return delta1

    def ajusta3(self, h2, eta):
        for i in range(0, self.n_oculta2):
            for j in range(0, self.n_saida):
                self.w3[i][j] = self.w3[i][j] + eta * self.delta3[j] * h2[i]
    
    def ajusta2(self, h1, eta):
        for i in range(0, self.n_oculta1):
            for j in range(1, self.n_oculta2):
                self.w2[i][j] = self.w2[i][j] + eta * self.delta2[j] * h1[i]
        
    def ajusta1(self, x, eta):
        for i in range(0, self.n_entrada):
            for j in range(1, self.n_oculta1):
                self.w1[i][j] = self.w1[i][j] + eta * self.delta1[j] * x[i]
                

    def treinar(self, X, Y, eta, max_interacoes, erro_aceitavel, verbose=True):
        self.w1, self.w2, self.w3 = self.inicializa_pesos(self.n_entrada, self.n_oculta1, self.n_oculta2 , self.n_saida)
        erro_max = erro_aceitavel * 2
        erro = 0.0
        i, m = 1, 0

        while erro_max > erro_aceitavel and i <= max_interacoes:
            if m == len(X): m = 0; erro = 0.0

            x = X[m]
            y = Y[m]

            '''h1 = self.ativacao(x, self.w1, self.n_entrada, self.n_oculta1-1)
            h1 = np.insert(h1, 0, self.bias)
            h2 = self.ativacao(h1, self.w2, self.n_oculta1, self.n_oculta2-1)
            h2 = np.insert(h2, 0, self.bias)
            o = self.ativacao(h2, self.w3, self.n_oculta2, self.n_saida)'''
            h1 = self.intermediaria1(x)
            h2 = self.intermediaria2(h1)
            o = self.saida(h2)
    
            erro_saida = self.erro_saida(o, y)

            self.delta3 = self.erro3(o, y)
            self.delta2 = self.erro2(h2, self.delta3)
            self.delta1 = self.erro1(h1, self.delta2)

            self.ajusta3(h2, eta[2])
            self.ajusta2(h1, eta[1])
            self.ajusta1(x, eta[0])

            if verbose:
                print(f'Época: {i} - Padrão>> {m+1} - Erro: {erro_saida:.9f}')
            
            
            #if erro_saida > erro_max: erro_max = erro_saida
            erro += erro_saida 
            if m == len(X)-1:
                if erro < erro_aceitavel*len(X):
                    erro_max = erro
            i += 1
            m += 1

        self.erro_max = erro_max
        self.epocas = i - 1
        print(f'Épocas executadas: {i-1}')
        print(f'Erro máximo durante o treino: {erro_max:.9f}')
        #print(f'Último valor de erro: {erro_saida:.9f}')
        
        if verbose:
            print(f'\nTestando com os mesmos padrões usados no treinamento...')
            emq, acuracia = self.testar(X, Y)
            print(f'EMQ com os dados de treinamento: {emq:.9f}')
            print(f'Acurácia com os dados de treinamento: {acuracia:.2f}%')

    def testar(self, X, Y, verbose=False):
        Y_obtido = []
        erro_saida = []
        emq = 0.0
        for m in range(len(X)):
            x = X[m]
            y = Y[m]
            '''h1 = self.ativacao(x, self.w1, self.n_entrada, self.n_oculta1-1)
            h1 = np.insert(h1, 0, self.bias)
            h2 = self.ativacao(h1, self.w2, self.n_oculta1, self.n_oculta2-1)
            h2 = np.insert(h2, 0, self.bias)
            o = self.ativacao(h2, self.w3, self.n_oculta2, self.n_saida)'''
            h1 = self.intermediaria1(x)
            h2 = self.intermediaria2(h1)
            o = self.saida(h2)
            erro_saida.append(self.erro_saida(o, y))
            o_9 = [f'{y:.9f}' for y in o]
            if verbose:
                print(f'Padrão>> {m+1} - Calculado: {o_9[0]}, Desejado: {y}, Erro: {erro_saida[-1]:.9f}')
            Y_obtido.append(o)
        
        emq = np.sum(erro_saida) / len(X) # calcula o erro quadrático médio
        
        qtd_acertos, acuracia = self.avaliar(Y_obtido, Y, erro_saida)
        self.qtd_acertos = qtd_acertos
        if verbose:
            print(f'\nErro Médio Quadrático: {emq:.9f}')
            print(f'Quantidade de acertos: {qtd_acertos}')
            print(f'Quantidade de erros: {len(X)-qtd_acertos}')
            print(f'\nAcurácia: {acuracia:.2f}%')
            df_w1 = pd.DataFrame(self.w1[:,1:], columns=['w1_h'+str(i) for i in range(1, self.n_oculta1)], index=['entrada_'+str(i) for i in range(self.n_entrada)])
            #print(f'\nPesos Entrada/Oculta1:')
            #display(df_w1)
            df_w2 = pd.DataFrame(self.w2[:,1:], columns=['w2_h'+str(i) for i in range(1, self.n_oculta2)], index=['oculta1_'+str(i) for i in range(self.n_oculta1)])
            #print(f'\nPesos Oculta1/Oculta2:')
            #display(df_w2)
            df_w3 = pd.DataFrame(self.w3, columns=['w3_o'+str(i) for i in range(self.n_saida)], index=['oculta2_'+str(i) for i in range(self.n_oculta2)])
            #print(f'Pesos Oculta2/Saida:')
            #display(df_w3)
            
            Y_obtido_9 = [f'{y[0]:.9f}'  for y in Y_obtido]
            with open('resultado_' + self.nome_dt + '_4C.txt', 'w') as arquivo:
                arquivo.write(f'Cenário experimental com 4 camadas: {self.nome_dt}\n')
                for i in range(len(erro_saida)):
                    arquivo.write(f'\nPadrão>> {i+1} - Calculado: {Y_obtido_9[i]}, Desejado: {Y[i]}, Erro: {erro_saida[i]:.9f}')
                arquivo.write(f'\n\nErro Médio Quadrático: {emq:.9f}')
                arquivo.write(f'\n\nAcurácia: {acuracia:.2f}%')
                arquivo.write(f'\n\nPesos Entrada/Oculta1:\n\n')
                arquivo.write(df_w1.to_string())
                arquivo.write(f'\n\nPesos Oculta1/Oculta2:\n\n')
                arquivo.write(df_w2.to_string())
                arquivo.write(f'\n\nPesos Oculta/Saida:\n\n')
                arquivo.write(df_w3.to_string())

        return emq, acuracia

    # Acurácia por aproximação
    def avaliar(self, Y_obtido, Y, erros):
        qtd_acertos = erro_total = 0
        for i in range(len(Y_obtido)):
            if Y[i] == 0.0 and Y_obtido[i] >= 0.0 and Y_obtido[i] < 0.5:
                qtd_acertos += 1
                erro_total += erros[i]
            elif Y[i] == 1.0 and Y_obtido[i] >= 0.5 and Y_obtido[i] <= 1.0:
                qtd_acertos += 1
                erro_total += erros[i]
        acuracia = ((qtd_acertos / len(Y_obtido)) - erro_total) * 100
        return qtd_acertos, acuracia

### Modelagem do conjunto treinamento

In [6]:
def conjunto_treinamento(dataset):
    bias = 0.1
    X = np.insert(dataset[:, :-1], 0, bias, axis=1) # Pegando os previsores e inserindo bias
    Y = dataset[:, -1] # Pegando as classes
    return X, Y

### Grid

In [15]:
def grid(n_entrada, x_treino, y_treino, x_teste, y_teste, dt_nome, NINT1, NINT2, TAP1, TAP2):
    rede = []
    f = open('grid_hould_out_' + dt_nome + '_4C.csv', 'w', newline='', encoding='utf-8')
    arquivo = csv.writer(f)
    arquivo.writerow(['Neurônios_C1','Neurônios_C2','Neurônios_saída','Taxa_aprend1','Taxa_aprend2','Épocas','Erro_max','Acertos','Erros','EMQ','Acurácia'])
    for nint1 in NINT1:
        for nint2 in NINT2:
            for tap1 in TAP1:
                for tap2 in TAP2:
                    print(f'Neurônios.C1: {nint1} - Neurônios.C2: {nint2} - Taxa.C1: {tap1} - Taxa.C2: {tap2}')
                    rede.append(MLP(n_entrada, [nint1, nint2], n_saida, aleatorio_uniforme, sigmoid, dt_nome))
                    rede[-1].treinar(x_treino, y_treino, [tap1, tap2, tap1], max_interacoes, erro_aceitavel, verbose=False)
                    emq, acc = rede[-1].testar(x_teste, y_teste, verbose=False)
                    epocas = rede[-1].epocas
                    erro_max = f'{rede[-1].erro_max:.9f}'
                    qtd_acertos = rede[-1].qtd_acertos
                    qtd_erros = len(x_teste) - qtd_acertos
                    emq = f'{emq:.9f}'
                    acc = f'{acc:.2f}%'
                    print(f'EMQ com os dados de treinamento: {emq}')
                    print(f'Acurácia com os dados de treinamento: {acc}')
                    print(f'')
                    arquivo.writerow([nint1,nint2,n_saida,tap1,tap2,epocas,erro_max,qtd_acertos,qtd_erros,emq,acc])
    f.close()

### Caso 01 - AND

In [8]:
n_entrada = 3   # Número de entradas + bias
n_oculta = [5, 17]    # Número de neurônios na camada oculta + bias
n_saida = 1     # Número de neurônios na camada de saída
tx_aprend = [0.90, 0.90, 0.90]
max_interacoes = 60000
erro_aceitavel = 0.0001

x_treinoAND, y_treinoAND = conjunto_treinamento(dt_AND)

rede = MLP(n_entrada, n_oculta, n_saida, aleatorio_uniforme, sigmoid, 'AND')
rede.treinar(x_treinoAND, y_treinoAND, tx_aprend, max_interacoes, erro_aceitavel, verbose=False)
print(f'')
emq, acc = rede.testar(x_treinoAND, y_treinoAND, verbose=True)

Épocas executadas: 47444
Erro máximo durante o treino: 0.000099994

Padrão>> 1 - Calculado: 0.000894097, Desejado: 0, Erro: 0.000000400
Padrão>> 2 - Calculado: 0.005641332, Desejado: 0, Erro: 0.000015912
Padrão>> 3 - Calculado: 0.006350042, Desejado: 0, Erro: 0.000020162
Padrão>> 4 - Calculado: 0.988732634, Desejado: 1, Erro: 0.000063477

Erro Médio Quadrático: 0.000024988
Quantidade de acertos: 4
Quantidade de erros: 0

Acurácia: 99.99%


### Caso 02 - OR

In [9]:
n_entrada = 3   # Número de entradas + bias
n_oculta = [5, 17]    # Número de neurônios na camada oculta + bias
n_saida = 1     # Número de neurônios na camada de saída
tx_aprend = [0.90, 0.90, 0.90]
max_interacoes = 60000
erro_aceitavel = 0.0001

x_treinoOR, y_treinoOR = conjunto_treinamento(dt_OR)

rede = MLP(n_entrada, n_oculta, n_saida, aleatorio_uniforme, sigmoid, 'OR')
rede.treinar(x_treinoOR, y_treinoOR, tx_aprend, max_interacoes, erro_aceitavel, verbose=False)
print(f'')
emq, acc = rede.testar(x_treinoOR, y_treinoOR, verbose=True)

Épocas executadas: 37060
Erro máximo durante o treino: 0.000099993

Padrão>> 1 - Calculado: 0.008186068, Desejado: 0, Erro: 0.000033506
Padrão>> 2 - Calculado: 0.992222900, Desejado: 1, Erro: 0.000030242
Padrão>> 3 - Calculado: 0.992525732, Desejado: 1, Erro: 0.000027932
Padrão>> 4 - Calculado: 0.995930877, Desejado: 1, Erro: 0.000008279

Erro Médio Quadrático: 0.000024990
Quantidade de acertos: 4
Quantidade de erros: 0

Acurácia: 99.99%


### Caso 03 - XOR

In [10]:
n_entrada = 3   # Número de entradas + bias
n_oculta = [5, 17]    # Número de neurônios na camada oculta + bias
n_saida = 1     # Número de neurônios na camada de saída
tx_aprend = [0.90, 0.90, 0.90]
max_interacoes = 60000
erro_aceitavel = 0.0001

x_treinoXOR, y_treinoXOR = conjunto_treinamento(dt_XOR)

rede = MLP(n_entrada, n_oculta, n_saida, aleatorio_uniforme, sigmoid, 'XOR')
rede.treinar(x_treinoXOR, y_treinoXOR, tx_aprend, max_interacoes, erro_aceitavel, verbose=False)
print(f'')
emq, acc = rede.testar(x_treinoXOR, y_treinoXOR, verbose=True)

Épocas executadas: 60000
Erro máximo durante o treino: 0.000189024

Padrão>> 1 - Calculado: 0.012683773, Desejado: 0, Erro: 0.000080439
Padrão>> 2 - Calculado: 0.990982549, Desejado: 1, Erro: 0.000040657
Padrão>> 3 - Calculado: 0.988737155, Desejado: 1, Erro: 0.000063426
Padrão>> 4 - Calculado: 0.002983683, Desejado: 0, Erro: 0.000004451

Erro Médio Quadrático: 0.000047243
Quantidade de acertos: 4
Quantidade de erros: 0

Acurácia: 99.98%


### Caso 04 - Tabela Lógica

In [11]:
n_entrada = 4   # Número de entradas + bias
n_oculta = [5, 17]    # Número de neurônios na camada oculta + bias
n_saida = 1     # Número de neurônios na camada de saída
tx_aprend = [0.90, 0.90, 0.90]
max_interacoes = 60000
erro_aceitavel = 0.0001

x_treinoL, y_treinoL = conjunto_treinamento(dt_LOGICA)

rede = MLP(n_entrada, n_oculta, n_saida, aleatorio_uniforme, sigmoid, 'Tabela_Lógica')
rede.treinar(x_treinoL, y_treinoL, tx_aprend, max_interacoes, erro_aceitavel, verbose=False)
print(f'')
emq, acc = rede.testar(x_treinoL, y_treinoL, verbose=True)

Épocas executadas: 60000
Erro máximo durante o treino: 0.000385109

Padrão>> 1 - Calculado: 0.019764259, Desejado: 0, Erro: 0.000195313
Padrão>> 2 - Calculado: 0.990594289, Desejado: 1, Erro: 0.000044234
Padrão>> 3 - Calculado: 0.990378448, Desejado: 1, Erro: 0.000046287
Padrão>> 4 - Calculado: 0.000679094, Desejado: 0, Erro: 0.000000231
Padrão>> 5 - Calculado: 0.000699789, Desejado: 0, Erro: 0.000000245
Padrão>> 6 - Calculado: 0.990122616, Desejado: 1, Erro: 0.000048781
Padrão>> 7 - Calculado: 0.990047083, Desejado: 1, Erro: 0.000049530
Padrão>> 8 - Calculado: 0.000107134, Desejado: 0, Erro: 0.000000006

Erro Médio Quadrático: 0.000048078
Quantidade de acertos: 8
Quantidade de erros: 0

Acurácia: 99.96%


### Grid nos casos

In [16]:
n_entrada1 = 3          # Número de entradas + bias
n_entrada2 = 4
n_saida = 1             # Número de neurônios na camada de saída
NINT1 = [5, 17]
NINT2 = [5, 17]
TAP1 = [0.90, 0.30]
TAP2 = [0.90, 0.30]
max_interacoes = 60000
erro_aceitavel = 0.0001

In [17]:
#Grid na base AND
grid(n_entrada1, x_treinoAND, y_treinoAND, x_treinoAND, y_treinoAND, 'AND', NINT1, NINT2, TAP1, TAP2)

Neurônios.C1: 5 - Neurônios.C2: 5 - Taxa.C1: 0.9 - Taxa.C2: 0.9
Épocas executadas: 29380
Erro máximo durante o treino: 0.000099988
EMQ com os dados de treinamento: 0.000024983
Acurácia com os dados de treinamento: 99.99%

Neurônios.C1: 5 - Neurônios.C2: 5 - Taxa.C1: 0.9 - Taxa.C2: 0.3
Épocas executadas: 53568
Erro máximo durante o treino: 0.000099995
EMQ com os dados de treinamento: 0.000024992
Acurácia com os dados de treinamento: 99.99%

Neurônios.C1: 5 - Neurônios.C2: 5 - Taxa.C1: 0.3 - Taxa.C2: 0.9
Épocas executadas: 60000
Erro máximo durante o treino: 0.000144076
EMQ com os dados de treinamento: 0.000036012
Acurácia com os dados de treinamento: 99.99%

Neurônios.C1: 5 - Neurônios.C2: 5 - Taxa.C1: 0.3 - Taxa.C2: 0.3
Épocas executadas: 60000
Erro máximo durante o treino: 0.000154469
EMQ com os dados de treinamento: 0.000038606
Acurácia com os dados de treinamento: 99.98%

Neurônios.C1: 5 - Neurônios.C2: 17 - Taxa.C1: 0.9 - Taxa.C2: 0.9
Épocas executadas: 47444
Erro máximo durante o 

In [18]:
#Grid na base OR
grid(n_entrada1, x_treinoOR, y_treinoOR, x_treinoOR, y_treinoOR, 'OR', NINT1, NINT2, TAP1, TAP2)

Neurônios.C1: 5 - Neurônios.C2: 5 - Taxa.C1: 0.9 - Taxa.C2: 0.9
Épocas executadas: 46832
Erro máximo durante o treino: 0.000099991
EMQ com os dados de treinamento: 0.000024993
Acurácia com os dados de treinamento: 99.99%

Neurônios.C1: 5 - Neurônios.C2: 5 - Taxa.C1: 0.9 - Taxa.C2: 0.3
Épocas executadas: 47300
Erro máximo durante o treino: 0.000099999
EMQ com os dados de treinamento: 0.000024995
Acurácia com os dados de treinamento: 99.99%

Neurônios.C1: 5 - Neurônios.C2: 5 - Taxa.C1: 0.3 - Taxa.C2: 0.9
Épocas executadas: 60000
Erro máximo durante o treino: 0.000316403
EMQ com os dados de treinamento: 0.000079090
Acurácia com os dados de treinamento: 99.97%

Neurônios.C1: 5 - Neurônios.C2: 5 - Taxa.C1: 0.3 - Taxa.C2: 0.3
Épocas executadas: 60000
Erro máximo durante o treino: 0.000265454
EMQ com os dados de treinamento: 0.000066353
Acurácia com os dados de treinamento: 99.97%

Neurônios.C1: 5 - Neurônios.C2: 17 - Taxa.C1: 0.9 - Taxa.C2: 0.9
Épocas executadas: 37060
Erro máximo durante o 

In [19]:
#Grid na base XOR
grid(n_entrada1, x_treinoXOR, y_treinoXOR, x_treinoXOR, y_treinoXOR, 'XOR', NINT1, NINT2, TAP1, TAP2)

Neurônios.C1: 5 - Neurônios.C2: 5 - Taxa.C1: 0.9 - Taxa.C2: 0.9
Épocas executadas: 60000
Erro máximo durante o treino: 0.000200000
EMQ com os dados de treinamento: 0.083588555
Acurácia com os dados de treinamento: 65.31%

Neurônios.C1: 5 - Neurônios.C2: 5 - Taxa.C1: 0.9 - Taxa.C2: 0.3
Épocas executadas: 60000
Erro máximo durante o treino: 0.000200000
EMQ com os dados de treinamento: 0.083678912
Acurácia com os dados de treinamento: 65.32%

Neurônios.C1: 5 - Neurônios.C2: 5 - Taxa.C1: 0.3 - Taxa.C2: 0.9
Épocas executadas: 60000
Erro máximo durante o treino: 0.000200000
EMQ com os dados de treinamento: 0.083383712
Acurácia com os dados de treinamento: 64.35%

Neurônios.C1: 5 - Neurônios.C2: 5 - Taxa.C1: 0.3 - Taxa.C2: 0.3
Épocas executadas: 60000
Erro máximo durante o treino: 0.000200000
EMQ com os dados de treinamento: 0.083618561
Acurácia com os dados de treinamento: 64.33%

Neurônios.C1: 5 - Neurônios.C2: 17 - Taxa.C1: 0.9 - Taxa.C2: 0.9
Épocas executadas: 60000
Erro máximo durante o 

In [20]:
#Grid na base Lógica
grid(n_entrada2, x_treinoL, y_treinoL, x_treinoL, y_treinoL, 'Tabela_Lógica', NINT1, NINT2, TAP1, TAP2)

Neurônios.C1: 5 - Neurônios.C2: 5 - Taxa.C1: 0.9 - Taxa.C2: 0.9
Épocas executadas: 60000
Erro máximo durante o treino: 0.000278623
EMQ com os dados de treinamento: 0.000034810
Acurácia com os dados de treinamento: 99.97%

Neurônios.C1: 5 - Neurônios.C2: 5 - Taxa.C1: 0.9 - Taxa.C2: 0.3
Épocas executadas: 60000
Erro máximo durante o treino: 0.000284449
EMQ com os dados de treinamento: 0.000035535
Acurácia com os dados de treinamento: 99.97%

Neurônios.C1: 5 - Neurônios.C2: 5 - Taxa.C1: 0.3 - Taxa.C2: 0.9
Épocas executadas: 60000
Erro máximo durante o treino: 0.000200000
EMQ com os dados de treinamento: 0.000165744
Acurácia com os dados de treinamento: 99.87%

Neurônios.C1: 5 - Neurônios.C2: 5 - Taxa.C1: 0.3 - Taxa.C2: 0.3
Épocas executadas: 60000
Erro máximo durante o treino: 0.000200000
EMQ com os dados de treinamento: 0.000131975
Acurácia com os dados de treinamento: 99.89%

Neurônios.C1: 5 - Neurônios.C2: 17 - Taxa.C1: 0.9 - Taxa.C2: 0.9
Épocas executadas: 60000
Erro máximo durante o 